# Variables of Interest Generation

In [8]:
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk import corpus
import pandas as pd
import time
import json

# english words dataset from https://www.kaggle.com/datasets/bwandowando/479k-english-words
words_dictionary = json.load(open('../data/words_dictionary.json'))

def check_if_word_exists(word, words_dictionary):
    if words_dictionary.get(word.lower()) is None:
        return False
    return True

df = pd.read_csv('../data/movies_cleaned.csv')

sia = SentimentIntensityAnalyzer()

# for getting the remaining time estimate
start_time = time.time()

for index, row in df.iterrows():
    print(f'Processing row {index} of {len(df)}')

    # generate polarity scores
    title = row['title']
    title_sentiment = sia.polarity_scores(title)
    df.at[index, 'title_sentiment'] = title_sentiment['compound']

    overview = str(row['overview'])
    overview_sentiment = sia.polarity_scores(overview)
    df.at[index, 'overview_sentiment'] = overview_sentiment['compound']

    # generate length features
    title_length = len(title)
    df.at[index, 'title_length'] = title_length
    
    num_words_in_title = len(title.split())
    df.at[index, 'num_words_in_title'] = num_words_in_title

    title_words = title.split()
    average_title_word_length = sum(len(word) for word in title_words) / len(title_words)
    df.at[index, 'average_title_word_length'] = average_title_word_length

    # check if there is a word in the title that does not exist in the dictionary
    # first strip out  title punctuation, numbers, and stopwords like "of, the, a, etc."
    title = ''.join([char for char in title if char.isalpha() or char.isspace()])
    title_words_not_in_dict = [word for word in title.split() if word.lower() not in corpus.stopwords.words("english")]
    title_words_not_in_dict = [word for word in title_words_not_in_dict if not check_if_word_exists(word, words_dictionary)]
    df.at[index, 'has_made_up_word'] = len(title_words_not_in_dict) > 0
    df.at[index, 'made_up_word_count'] = len(title_words_not_in_dict)

    
df.to_csv('../data/movies_with_vars.csv', index=False)
df.describe()

Processing row 0 of 6507
Processing row 1 of 6507
Processing row 2 of 6507
Processing row 3 of 6507
Processing row 4 of 6507
Processing row 5 of 6507
Processing row 6 of 6507
Processing row 7 of 6507
Processing row 8 of 6507
Processing row 9 of 6507
Processing row 10 of 6507
Processing row 11 of 6507
Processing row 12 of 6507
Processing row 13 of 6507
Processing row 14 of 6507
Processing row 15 of 6507
Processing row 16 of 6507
Processing row 17 of 6507
Processing row 18 of 6507
Processing row 19 of 6507
Processing row 20 of 6507
Processing row 21 of 6507
Processing row 22 of 6507
Processing row 23 of 6507
Processing row 24 of 6507
Processing row 25 of 6507
Processing row 26 of 6507
Processing row 27 of 6507
Processing row 28 of 6507
Processing row 29 of 6507
Processing row 30 of 6507
Processing row 31 of 6507
Processing row 32 of 6507
Processing row 33 of 6507
Processing row 34 of 6507
Processing row 35 of 6507
Processing row 36 of 6507
Processing row 37 of 6507
Processing row 38 of 6

,id,popularity,vote_average,title_sentiment,overview_sentiment,title_length,num_words_in_title,average_title_word_length,made_up_word_count
count,6.507000e+03,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000
mean,2.230877e+05,25.875517,6.455688,-0.016061,-0.115678,16.223605,2.887659,5.409678,0.177501
std,3.012924e+05,11.720481,0.794394,0.244928,0.627946,9.645656,1.780141,1.818371,0.418226
min,5.000000e+00,9.950000,2.895000,-0.937500,-0.987700,1.000000,1.000000,1.000000,0.000000
25%,1.005200e+04,17.486000,5.975000,0.000000,-0.718400,10.000000,2.000000,4.000000,0.000000
50%,3.282300e+04,22.174000,6.492000,0.000000,-0.202300,14.000000,2.000000,5.000000,0.000000
75%,4.088845e+05,31.149000,7.000000,0.000000,0.461050,20.000000,4.000000,6.000000,0.000000
max,1.237835e+06,63.605000,8.675000,0.891000,0.995800,104.000000,20.000000,17.000000,4.000000
